In [1]:
# 2022 COMP90024 Group 33 

# Team members:

# Ke Yang (Student ID: 1219623) - city: Anhui

# Yimeng Liu (Student ID: 1074206) - city: Guangdong

# Jintong Liu (Student ID: 1074498) - city: Hebei

# Keang Xu (Student ID: 1008807) - city: Hubei

# Xinwei Qian (Student ID: 1068271) - city: Jiangsu


from distutils import text_file
from cloudant.client import CouchDB
from cloudant.view import View
from cloudant.result import Result
import json
from collections import Counter
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
import re


def textProcess(text):
    text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', " ", text)  # remove URL
    text = text.lower() # lowercase the text
#         text = re.sub(r'@[^ ]+', ' ', text) # replace mention
    # text = re.sub(r'#', '', text) # remove hashtag
    text = re.sub(r'([a-z])\1{2,}', r'\1', text) # character normalization
    text = re.sub(r'[^a-z#@0-9 ]', ' ', text) # remove non-letter characters
    text = text.strip()
    return text

def couchdb_init():
    USERNAME = 'admin'
    PASSWORD = 'comp90024'
    client = CouchDB(USERNAME, PASSWORD, url='http://127.0.0.1:5984', connect=True)
    return client

def calculate_city_score(text):

    text = textProcess(text)
    sia = SentimentIntensityAnalyzer()
    score = sia.polarity_scores(text)
    return score

def hashtags_analysis():
    # count hashtags
    citys = {}
    dbname = ["db_darwin", "db_melbourne", "db_sydney"]
    design_hashtag_doc= '''
    {
        "_id" : "_design/text",
        "language": "javascript",
        "views" : {
          "new_view" : {
            "map": "function(doc){emit(doc.text)}",  
            "reduce" : "_count"}
        }
    }
    '''
    client = couchdb_init()
    compound_score = []
    neg_score = []
    neu_score = []
    pos_score = []
    
    for city in dbname:
        citydb = client[city]
        json_data = json.loads(design_hashtag_doc, strict=False)
        client = couchdb_init()
        
        text_dict = {}
 
        if not json_data['_id'] in citydb:
            citydb.create_document(json_data)

        create_view = View(citydb['_design/text'], 'new_view')

        with create_view.custom_result(group=True) as results:
            i = 0
            for result in results:
                text_dict[i]= result['key']
                i+=1
                
        c_score_sum = 0
        neg_score_sum = 0
        neu_score_sum = 0
        pos_score_sum = 0
            
        sia = SentimentIntensityAnalyzer()
        results = {}
        for content in text_dict.values():
            text = textProcess(content)
            score_all_dict = sia.polarity_scores(text)
            c_score_sum = c_score_sum + score_all_dict['compound']
            neg_score_sum = neg_score_sum + score_all_dict['neg']
            neu_score_sum = neu_score_sum + score_all_dict['neu']
            pos_score_sum = pos_score_sum + score_all_dict['pos']

        ave_compound_score = c_score_sum/len(text_dict.values())
        compound_score.append(ave_compound_score)
        ave_neg_score = neg_score_sum/len(text_dict.values())
        neg_score.append(ave_neg_score)
        ave_neu_score = neu_score_sum/len(text_dict.values())
        neu_score.append(ave_neu_score)
        ave_pos_score = pos_score_sum/len(text_dict.values()) 
        pos_score.append(ave_pos_score)
        
    results["compound"] = compound_score
    results["neg"] = neg_score
    results["neu"] = neu_score
    results["pos"] = pos_score
    
    return results
           

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/ccc/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
hashtags_analysis()

{'compound': [0.10539101471727336, 0.11302384982005363, 0.11692170441531825],
 'neg': [0.04990085205267235, 0.05408899912459882, 0.04806893370657836],
 'neu': [0.7929465530596437, 0.7832408650261056, 0.784472642829884],
 'pos': [0.12306816421378772, 0.1296324611743356, 0.12356024939559876]}